In [ ]:
!pip install gradio pandas numpy mlxtend

In [ ]:
import pandas as pd
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori, association_rules

# Sample dataset (or upload your own)
data = [
    ["milk", "bread", "eggs"],
    ["bread", "sugar"],
    ["milk", "sugar"],
    ["bread", "eggs", "sugar"],
    ["milk", "bread", "eggs", "sugar"],
    ["milk", "bread"],
    ["bread", "eggs"],
    ["milk", "eggs"],
    ["milk", "bread", "eggs", "butter"],
    ["bread", "sugar", "butter"]
]

# Convert to one-hot encoded format
te = TransactionEncoder()
te_data = te.fit_transform(data)
df = pd.DataFrame(te_data, columns=te.columns_)
print(df.head())

   bread  butter   eggs   milk  sugar
0   True   False   True   True  False
1   True   False  False  False   True
2  False   False  False   True   True
3   True   False   True  False   True
4   True   False   True   True   True


In [ ]:
# Find frequent itemsets (min_support = 0.2)
frequent_itemsets = apriori(df, min_support=0.2, use_colnames=True)
print("\nFrequent Itemsets:")
print(frequent_itemsets)

# Generate association rules (min_confidence = 0.7)
rules = association_rules(frequent_itemsets, metric="confidence", min_threshold=0.7)
print("\nAssociation Rules:")
print(rules[['antecedents', 'consequents', 'support', 'confidence', 'lift']])


Frequent Itemsets:
    support              itemsets
0       0.8               (bread)
1       0.2              (butter)
2       0.6                (eggs)
3       0.6                (milk)
4       0.5               (sugar)
5       0.2       (bread, butter)
6       0.5         (bread, eggs)
7       0.4         (milk, bread)
8       0.4        (sugar, bread)
9       0.4          (milk, eggs)
10      0.2         (sugar, eggs)
11      0.2         (milk, sugar)
12      0.3   (milk, bread, eggs)
13      0.2  (sugar, bread, eggs)

Association Rules:
     antecedents consequents  support  confidence      lift
0       (butter)     (bread)      0.2    1.000000  1.250000
1         (eggs)     (bread)      0.5    0.833333  1.041667
2        (sugar)     (bread)      0.4    0.800000  1.000000
3  (milk, bread)      (eggs)      0.3    0.750000  1.250000
4   (milk, eggs)     (bread)      0.3    0.750000  0.937500
5  (sugar, eggs)     (bread)      0.2    1.000000  1.250000


In [ ]:
import gradio as gr

def market_basket_analysis(min_support, min_confidence):
    # Run Apriori
    frequent_itemsets = apriori(df, min_support=min_support/100, use_colnames=True)
    rules = association_rules(frequent_itemsets, metric="confidence", min_threshold=min_confidence/100)

    # Format results
    itemsets_str = "=== Frequent Itemsets ===\n"
    for _, row in frequent_itemsets.iterrows():
        itemsets_str += f"{', '.join(row['itemsets'])} (Support: {row['support']:.3f})\n"

    rules_str = "\n=== Association Rules ===\n"
    for _, row in rules.iterrows():
        rules_str += (
            f"{', '.join(row['antecedents'])} → {', '.join(row['consequents'])}\n"
            f"  Support: {row['support']:.3f}, Confidence: {row['confidence']:.3f}, Lift: {row['lift']:.3f}\n\n"
        )

    return itemsets_str + rules_str

# Gradio UI
iface = gr.Interface(
    fn=market_basket_analysis,
    inputs=[
        gr.Slider(1, 50, value=20, label="Minimum Support (%)"),
        gr.Slider(1, 100, value=70, label="Minimum Confidence (%)")
    ],
    outputs=gr.Textbox(label="Market Basket Analysis Results", lines=20),
    title="Market Basket Analysis with Apriori Algorithm",
    description="Adjust support and confidence to find product associations."
)

iface.launch(debug=True)

It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://c586c0a5efac769590.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://c586c0a5efac769590.gradio.live
